In [6]:
def check(a):
    if 48 <= ord(a) <= 57 or 97 <= ord(a) <= 122 or 65 <= ord(a) <= 90 or a == "#":
        return True
    else:
        return False


def token_generate(regex): #((a|b)*.a.b.b).#
    stack = []
    i = 0
    while i < len(regex):
        if regex[i] == '(':
            j = i
            paren_checker = 0
            while j < len(regex):
                if regex[j] == '(':
                    paren_checker += 1
                elif regex[j] == ')':
                    paren_checker -= 1
                    if paren_checker == 0:
                        break
                j += 1
            stack.append(token_generate(regex[i + 1:j]))
            i = j
        elif regex[i] == '|':
            stack.append('|')
        elif regex[i] == '*':
            stack[-1] = [stack[-1], '*'] 
        elif regex[i] == '+':
            stack[-1] = [stack[-1], '+']
        elif regex[i] == '?':
            stack[-1] = [stack[-1], '?']
        else:
            stack.append(regex[i])
        i += 1

    while len(stack) > 1:
        left = stack.pop(0)
        operator = stack.pop(0)
        right = stack.pop(0)
        stack.insert(0, [left, operator, right])
    print(stack[0])
    return stack[0]


def modify(a):
    a = '(' + a + ')#'
    b = ''
    prio_1 = ['(', ')']
    prio_2 = ['*', '+', '?']
    prio_3 = ['|', '.']
    prev = None
    count = 0
    for i in a:
        if count == 0: #(a|b)*abb
            prev = i
            b += i
            count += 1
        elif (i == '(' and prev in prio_2) or (check(i) == True and prev in prio_2) or (
                check(i) == True and check(prev) == True) or (check(prev) == True and i == '(') or (
                prev == ')' and check(i) == True) or (prev == ')' and i == "("):
            b += '.' + i
            prev = i
            count += 1
        else:
            b += i
            count += 1
            prev = i
    return b


class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.nullable = False
        self.first_pos = []
        self.last_pos = []

    def detail(self):
        print("Root:", self.data, end=", ")
        if self.left != None:
            print("Left:", self.left.data, end=", ")
        if self.right != None:
            print("Right:", self.right.data, end=", ")
        print(
            f"Nullable status: {self.nullable}, Firstpos: {self.first_pos}, Lastpos: {self.last_pos}")


def starFollow(n):
    for x in list(n.left.first_pos):
        if x not in follow.keys():
            follow[x] = n.left.last_pos
        else:
            for y in list(n.left.last_pos):
                if y not in follow[x]:
                    follow[x].append(y)


def catFollow(node):
    for x in list(node.left.last_pos):
        if x not in follow.keys():
            follow[x] = node.right.first_pos
        else:
            for y in list(node.right.first_pos):
                if y not in follow[x]:
                    follow[x].append(y)


def createTree(t):
    global follow
    if type(t[0]) == list: #((a|b)*.a.b.b).#
        left = createTree(t[0]) #[[[[[['a', '|', 'b'], '*'], '.', 'a'], '.', 'b'], '.', 'b'], '.', '#']
    else:
        left = leaf(t[0])

    if len(t) == 2:
        data = t[1]
        n = Node(data)
        n.left = left

        if data == "*" or data == "?":
            n.nullable = True
        n.first_pos = n.left.first_pos
        n.last_pos = n.left.last_pos
        if data == "*" or data == "+":
            starFollow(n)
        n.detail()
        return n

    elif len(t) == 3:
        data = t[1]
        n = Node(data)
        n.left = left

        if type(t[2]) == list:
            n.right = createTree(t[2])
        else:
            n.right = leaf(t[2])

        if data == ".":
            if n.left.nullable:
                n.first_pos = n.left.first_pos + n.right.first_pos
            else:
                n.first_pos = n.left.first_pos
            if n.right.nullable:
                n.last_pos = n.left.last_pos + n.right.last_pos
            else:
                n.last_pos = n.right.last_pos
            if n.left.nullable and n.right.nullable:
                n.nullable=True
            catFollow(n)
        else:
            n.first_pos = n.left.first_pos + n.right.first_pos
            n.last_pos = n.left.last_pos + n.right.last_pos
        n.detail()
        return n


def leaf(x):
    global count
    count += 1
    c = Node(x)
    c.first_pos = [count]
    c.last_pos = [count]
    return c


def DState_Table(root, symbol, sequence, follow):
    start = tuple(root.first_pos)
    stack = [start]
    transition = {}
    while stack:
        x = stack.pop(0)
        transition[x] = {}
        for alpha in symbol:
            for n in x:
                # print(sequence)
                if n >= len(sequence) or len(sequence)==0:
                    transition[x]["accept"] = True
                    break
                if sequence[n] == alpha:
                    if alpha not in transition[x].keys():
                        transition[x][alpha] = []
                    transition[x][alpha] += tuple(follow[n])
                    transition[x][alpha] = set(transition[x][alpha])
                    transition[x][alpha] = tuple(transition[x][alpha])
                    transition[x]["accept"] = False
        for y in transition[x].values():
            if y==[]:
                continue
            if y not in transition.keys() and y not in [True, False]:
                stack.append(y)
        # print(transition)
    return transition


def DFA(dstate, string):
    state = list(dstate.keys())[0]
    accept = dstate[state]["accept"]
    for x in string:
        if x not in dstate[state].keys():
            return False
        state = tuple(dstate[state][x])
        # print(dstate, state)
        accept = dstate[state]["accept"]
    return accept


def driver(a="(a|b)*abb", string="ajhsbb"):
    global follow
    b = modify(a)
    symbol = []
    sequence = [0]
    for i in a:
        temp = check(i)
        if temp == True:
            sequence.append(i)
            if i not in symbol:
                symbol.append(i)

    t = token_generate(b)
    # print(t)
    root = createTree(t)
    print("FollowPos: ",follow)
    dstate = DState_Table(root, symbol, sequence, follow)
    result = DFA(dstate, string)
    print(f"RegEX: {a}, String: {string}, Acceptance: {result}")



while True:
    count = 0
    follow = {}
    print("**Click Enter to stop Input**")
    reg = input("Enter the Regex: ")
    if reg == "":
        break
    string = input("Now Enter the String: ")
    driver(reg, string)
    print()


**Click Enter to stop Input**
['a', '|', 'b']
[[[[['a', '|', 'b'], '*'], '.', 'a'], '.', 'a'], '.', 'b']
[[[[[['a', '|', 'b'], '*'], '.', 'a'], '.', 'a'], '.', 'b'], '.', '#']
Root: |, Left: a, Right: b, Nullable status: False, Firstpos: [1, 2], Lastpos: [1, 2]
Root: *, Left: |, Nullable status: True, Firstpos: [1, 2], Lastpos: [1, 2]
Root: ., Left: *, Right: a, Nullable status: False, Firstpos: [1, 2, 3], Lastpos: [3]
Root: ., Left: ., Right: a, Nullable status: False, Firstpos: [1, 2, 3], Lastpos: [4]
Root: ., Left: ., Right: b, Nullable status: False, Firstpos: [1, 2, 3], Lastpos: [5]
Root: ., Left: ., Right: #, Nullable status: False, Firstpos: [1, 2, 3], Lastpos: [6]
FollowPos:  {1: [1, 2, 3], 2: [1, 2, 3], 3: [4], 4: [5], 5: [6]}
RegEX: (a|b)*aab, String: abb, Acceptance: False

**Click Enter to stop Input**


KeyboardInterrupt: ignored